# <center>情報抽出</center>

In [101]:
# 使用するライブラリ
import pandas as pd
import glob

### データの読み込み

In [ ]:
file_paths = glob.glob("*.csv")

In [103]:
df_list = []
for file_path in file_paths:
    df_list.append(pd.read_csv(file_path,encoding="utf-8"))

In [104]:
genre = ["まぜそば","家系","二郎系","激辛","つけ麺"]
kakutyo_list = []
for df,g in zip(df_list,genre):
    df["genre"] = g
    kakutyo_list.append(df)

In [106]:
df=pd.concat(kakutyo_list)

In [108]:
df = df.sort_values("number")
df = df.reset_index()

### 各種情報の抽出

In [197]:
def description_analaysis(description,tdfk):
    # 各種情報を取得
    shop_data   = description[1].split("\n")
    check_list = 0
    shop_place = ""
    shop_link = ""
    shop_name = ""
    
    for data in shop_data[1:]:
        # データのお掃除
        if all(key not in data for key in ["■","#","★","◆"]) and len(shop_place) == 0:
            shop_place = data
            shop_place = shop_place.split("http")
            if len(shop_place) == 2:
                shop_link = "http"+shop_place[1]
            shop_place = shop_place[0]
        if "http" in data and len(shop_link) == 0:
            shop_link = data
    shop_name = shop_data[0]
    if len(shop_name) == 0:
        shop_name = shop_data[1]
        
    print(shop_name,shop_place,shop_link)
        
    return shop_name, shop_place,shop_link

In [199]:
def get_data(df):
    descriptions = df["description"].values.tolist()
    cnt = 0
    drop_list = []
    # 情報抽出できない動画を削除する
    for description in descriptions:
        # 概要欄の情報を整理
        description  = description.split("【本日のお店情報】")
        if len(description) == 1:
            description  = description[0].split("【本日のラーメン情報】")
            if len(description) == 1:
                description  = description[0].split("【本日のお店】")
                if len(description) == 1:
                    drop_list.append(cnt)
        cnt += 1
    print(len(df))
    df = df.drop(df.index[drop_list])
    print(len(df))

     
    # データの抽出
    descriptions = df["description"].values.tolist()
    susuru_comments = []
    shop_names=[]
    shop_places = []
    shop_links = []
    cnt = 0
    drop_list = []
    
    for description in descriptions:
        # 概要欄の情報を整理
        description  = description.split("【本日のお店情報】")
        if len(description) == 1:
            description  = description[0].split("【本日のラーメン情報】")
            if len(description) == 1:
                description  = description[0].split("【本日のお店】")         
        susuru_comment     = description[0].split("\n")[-1]
        # 各種情報を取得
        shop_name, shop_place,shop_link = description_analaysis(description,tdfk)
            
        if len(shop_name) == 0 or len(shop_place) == 0 or len(shop_link) == 0:
            drop_list.append(cnt)
            cnt+=1
            continue
        # リストに格納
        shop_names.append(shop_name)
        shop_places.append(shop_place)
        shop_links.append(shop_link)
        
        cnt += 1
    print(len(df),len(drop_list))
    df = df.drop(df.index[drop_list])
    # データフレームに格納
    df["shop_name"] = shop_names
    df["shop_place"] = shop_places
    df["shop_link"] = shop_links
    
    return df

In [203]:
df_extention = get_data(df)
df_extention = df_extention.drop(["Unnamed: 0","Unnamed: 0.1"], axis=1)

In [204]:
df_extention = df_extention.reset_index()

In [205]:
drop_list = ["channel","id","description"]
df_extention = df_extention.drop(drop_list, axis=1)

In [206]:
# 保存
df_extention.to_csv("保存するためのパス",encoding="utf-8",index=False)